In [1]:
import os

In [2]:
%pwd

'/home/himanshu/github_himanshu/end-to-end-ML-with-MLflow/research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/home/himanshu/github_himanshu/end-to-end-ML-with-MLflow'

In [5]:
from dataclasses import dataclass  # Import the dataclass decorator for creating data classes
from pathlib import Path  # Import the Path class for handling filesystem paths

@dataclass(frozen=True)  # Define the class as a dataclass and make it immutable with frozen=True
class ModelTrainerConfig:
    """
    Configuration class for model training.

    Attributes:
        root_dir (Path): The root directory where model training operations will take place.
        train_data_path (Path): The path to the training data.
        test_data_path (Path): The path to the testing data.
        model_name (str): The name of the model to be trained.
        alpha (float): The regularization strength parameter for the model.
        l1_ratio (float): The ElasticNet mixing parameter.
        target_column (str): The name of the target column in the dataset.
    """
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    alpha: float
    l1_ratio: float
    target_column: str


In [6]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH, schema_filepath=SCHEMA_FILE_PATH):
        """
        Initialize ConfigurationManager with paths to the configuration files.
        
        Args:
            config_filepath (Path): Path to the configuration YAML file.
            params_filepath (Path): Path to the parameters YAML file.
            schema_filepath (Path): Path to the schema YAML file.
        """
        self.config = read_yaml(config_filepath)  # Read the configuration file
        self.params = read_yaml(params_filepath)  # Read the parameters file
        self.schema = read_yaml(schema_filepath)  # Read the schema file

        create_directories([self.config.artifacts_root])  # Create the root directory for artifacts

    def get_model_trainer_config(self) -> ModelTrainerConfig:
        """
        Get the ModelTrainerConfig object from the configuration.
        
        Returns:
            ModelTrainerConfig: Configuration for model training.
        """
        config = self.config.model_trainer  # Get the model trainer configuration section
        params = self.params.ElasticNet  # Get the ElasticNet parameters
        schema = self.schema.TARGET_COLUMN  # Get the target column from the schema

        create_directories([config.root_dir])  # Ensure the root directory for model training exists

        # Create and return a ModelTrainerConfig object with the configuration details
        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            train_data_path=config.train_data_path,
            test_data_path=config.test_data_path,
            model_name=config.model_name,
            alpha=params.alpha,
            l1_ratio=params.l1_ratio,
            target_column=schema.name
        )

        return model_trainer_config


In [8]:
import pandas as pd
import os
from mlProject import logger
from sklearn.linear_model import ElasticNet
import joblib

[2024-06-13 14:44:38,670: INFO: utils: Note: NumExpr detected 12 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.]
[2024-06-13 14:44:38,671: INFO: utils: NumExpr defaulting to 8 threads.]


In [9]:
import pandas as pd  # Import pandas for data manipulation
from sklearn.linear_model import ElasticNet  # Import ElasticNet for regression
import joblib  # Import joblib for saving the model
import os  # Import os for operating system functionalities

class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        """
        Initialize the ModelTrainer class with a configuration.
        
        Args:
            config (ModelTrainerConfig): Configuration for model training.
        """
        self.config = config  # Store the configuration

    def train(self):
        """
        Train the ElasticNet model using the training data and save the model.
        """
        # Load the training and testing data from the specified paths
        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)

        # Split the data into features (X) and target (y) for training and testing
        train_x = train_data.drop([self.config.target_column], axis=1)
        test_x = test_data.drop([self.config.target_column], axis=1)
        train_y = train_data[[self.config.target_column]]
        test_y = test_data[[self.config.target_column]]

        # Initialize the ElasticNet model with the specified alpha and l1_ratio
        lr = ElasticNet(alpha=self.config.alpha, l1_ratio=self.config.l1_ratio, random_state=42)
        
        # Fit the model to the training data
        lr.fit(train_x, train_y)

        # Save the trained model to the specified path
        joblib.dump(lr, os.path.join(self.config.root_dir, self.config.model_name))


In [10]:
try:
    # Initialize ConfigurationManager with default file paths
    config = ConfigurationManager()
    
    # Retrieve model trainer configuration
    model_trainer_config = config.get_model_trainer_config()
    
    # Initialize ModelTrainer with the retrieved configuration
    model_trainer = ModelTrainer(config=model_trainer_config)
    
    # Train the model
    model_trainer.train()
except Exception as e:
    # Raise the exception if any error occurs
    raise e


[2024-06-13 14:44:39,335: INFO: common: YAML file: config/config.yaml loaded successfully]
[2024-06-13 14:44:39,337: INFO: common: YAML file: params.yaml loaded successfully]
[2024-06-13 14:44:39,339: INFO: common: YAML file: schema.yaml loaded successfully]
[2024-06-13 14:44:39,341: INFO: common: Created directory at: artifacts]
[2024-06-13 14:44:39,342: INFO: common: Created directory at: artifacts/model_trainer]
